# Creación Variables del Pasivo

In [1]:
# cuentas_out['prom_dias_desde_ult_trans'] = gp_cuentas['dias_desde_ultima_trans'].mean()
# cuentas_out['prom_antiguedad_cuenta'] = gp_cuentas['antiguedad'].mean()

In [1]:
import pandas as pd 
import numpy as np 


pd.set_option('max_columns',None)
#pd.set_option('max_rows',100)

from datetime import datetime, timedelta 
from dateutil import relativedelta

#pd.set_option('max_columns',False)
#pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
# path_in="s3://adl-refined-dev-popular/propension/raw_piloto/producto/pasivo/2018/"
# #path_in = "s3://adl-refined-dev-popular/data_orig/productos/pasivo/productos_pasivo_aaatr_M2020"
# path_out = "s3://adl-refined-dev-popular/data_orig/productos/pasivo/productos_pasivo_aaatr_M2020"


# for i in range(1,4):
    
#     if i<=9: var_aux = '0'+str(i)
#     else: var_aux = str(i)
        
#     data = pd.read_csv(path_in+var_aux+".csv",sep=",",encoding='utf_8',
#                         dtype={'id_cliente':'str','ID_CLIENTE':'str','nro_cuenta':str,'NRO_CUENTA':str})
    
#     print('Periodo 2020',var_aux)
#     print(data.shape)
    
#     data.to_csv(path_out+var_aux+"2.csv",sep="|",encoding='utf_8', index= False)
    
#     del data



In [4]:
def func_dates(tabla_final,date):
    if date == '201907':
        tabla_final['fecha_apertura'] = pd.to_datetime(tabla_final['fecha_apertura'], dayfirst= True)
        tabla_final['fecha_ultima_transaccion'] = pd.to_datetime(tabla_final['fecha_ultima_transaccion'], dayfirst= True)
    elif date in dates2:
        tabla_final['fecha_apertura'] = pd.to_datetime(tabla_final['fecha_apertura'], format='%d/%m/%Y')
        tabla_final['fecha_ultima_transaccion'] = pd.to_datetime(tabla_final['fecha_ultima_transaccion'], format='%d/%m/%Y')
    else:
        tabla_final['fecha_apertura'] = pd.to_datetime(tabla_final['fecha_apertura'], format='%Y%m%d',errors='coerce')
        tabla_final['fecha_ultima_transaccion'] = pd.to_datetime(tabla_final['fecha_ultima_transaccion'], format='%Y%m%d',errors='coerce')
        
    return tabla_final 

In [14]:
path_in="s3://adl-refined-dev-popular/data_orig/productos/pasivo/"


tabla_final = pd.read_csv(path_in+"productos_pasivo_aaatr_M202005.csv",sep=",",encoding='utf_8',
                    dtype={'id_cliente':'str','ID_CLIENTE':'str','nro_cuenta':str,'NRO_CUENTA':str})

In [15]:
tabla_final.head()

,SK_CLIENTE,ID_CLIENTE,TIPO_ID,COD_PRODUCTO,NRO_CUENTA,FECHA_ULTIMA_TRANSACCION,FECHA_APERTURA,ESTADO,SALDO,PROMEDIO1,PROMEDIO2,PROMEDIO3,PROMEDIO4,PROMEDIO5,PROMEDIO6,PERIODO
0,NaN,NaN,1.0,230.0,230360190284,20200203.0,20190220.0,Activa,"5851,93",5851,5851,5851,54667,112225,15450,202005.0
1,383846.0,176655242662434501,1.0,230.0,230269037354,20200528.0,20190311.0,Activa,"1429321,31",1294412,1139129,1014574,139444,102484,862390,202005.0
2,NaN,NaN,1.0,230.0,230098057532,20200601.0,20191210.0,Activa,"249025,22",139570,247248,127309,26741,27199,162030,202005.0
3,NaN,NaN,1.0,230.0,230040255697,20200514.0,20200504.0,Activa,"8251,12",20953,0,0,0,0,0,202005.0
4,1879480.0,628654886748163501,1.0,230.0,230364696526,20200529.0,20181213.0,Activa,"29279,33",60093,38222,19751,13324,9519,347747,202005.0


## Creando pasivos Total

In [5]:
path_in="s3://adl-refined-dev-popular/data_orig/productos/pasivo/"

str_dates = ['201801','201802','201803','201804','201805','201806','201807','201808','201809','201810','201811','201812',
            '201901','201902','201903','201904','201905','201906','201907','201908','201909','201910','201911','201912',
            '202001','202002','202003']

product_type_dict = {'1':'corriente', '2':'ahorro', '3':'cdt'}


dates1 = ['201904','201908','201909','201910','201911','201912','202001','202002','202003']
dates2 = ['201801','201802','201803','201804','201805','201806','201807','201808','201809','201810','201811','201812',
         '201901','201902','201903','201905','201906']

for ix, date in enumerate(str_dates):
    
    print('Periodo:',date)
    if ix == 0:
        tabla_final = pd.read_csv(path_in+"productos_pasivo_aaatr_M"+date+".csv",sep="|",encoding='utf_8',
                    dtype={'id_cliente':'str','ID_CLIENTE':'str','nro_cuenta':str,'NRO_CUENTA':str})
        tabla_final.columns = tabla_final.columns.str.lower()
        
        ## Cambiar tipo producto:
        tabla_final['periodo'] = int(date)
        tabla_final['tipo_producto'] = tabla_final['cod_producto'].apply(lambda x: str(x)[0]).map(product_type_dict)
        tabla_final = func_dates(tabla_final,date)
        
        print(tabla_final.shape)  
        
    else:
        
        tabla_temp = pd.read_csv(path_in+"productos_pasivo_aaatr_M"+date+".csv",sep="|",encoding='utf_8',
                    dtype={'id_cliente':'str','ID_CLIENTE':'str','nro_cuenta':str,'NRO_CUENTA':str})
        tabla_temp.columns = tabla_temp.columns.str.lower()
        
        ## Cambiar tipo producto:
        tabla_temp['periodo'] = int(date)
        tabla_temp['tipo_producto'] = tabla_temp['cod_producto'].apply(lambda x: str(x)[0]).map(product_type_dict)
        tabla_temp = func_dates(tabla_temp,date)
        
        print(tabla_temp.shape)
        
        tabla_final = pd.concat([tabla_final,tabla_temp], ignore_index=True)
        
        
        del tabla_temp
    
    

Periodo: 201801
(754596, 17)
Periodo: 201802
(766081, 17)
Periodo: 201803
(774864, 17)
Periodo: 201804
(781128, 17)
Periodo: 201805
(789077, 17)
Periodo: 201806
(790855, 17)
Periodo: 201807
(793081, 17)
Periodo: 201808
(799850, 17)
Periodo: 201809
(802923, 17)
Periodo: 201810
(802260, 17)
Periodo: 201811
(803016, 17)
Periodo: 201812
(802499, 17)
Periodo: 201901
(798135, 17)
Periodo: 201902
(795339, 17)
Periodo: 201903
(791846, 17)
Periodo: 201904
(794084, 17)
Periodo: 201905
(775217, 17)
Periodo: 201906
(778347, 17)
Periodo: 201907
(772755, 17)
Periodo: 201908
(765527, 17)
Periodo: 201909
(761364, 17)
Periodo: 201910
(755992, 17)
Periodo: 201911
(788036, 17)
Periodo: 201912
(776627, 17)
Periodo: 202001
(760165, 17)
Periodo: 202002
(749874, 17)
Periodo: 202003
(736635, 17)


In [6]:
tabla_final = tabla_final.drop_duplicates()

In [7]:
print(tabla_final.shape)
tabla_final.head()

(21034096, 17)


,sk_cliente,id_cliente,tipo_id,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,tipo_producto
0,2006882.0,538452294130750201,1,230,230066154014,2017-11-01,2017-11-01,Nueva,0,0,0,0,0,0,0,201801,ahorro
1,1643132.0,489152294130845901,1,230,230018022111,2017-12-15,2004-08-03,Activa,0,0,0,0,0,0,0,201801,ahorro
2,1951390.0,482452305350339702,1,230,230018022111,2017-12-15,2004-08-03,Activa,0,0,0,0,0,0,0,201801,ahorro
3,411977.0,186852309903613301,1,230,230029108743,2018-01-29,2009-04-28,Activa,227339,795929,1.01184e+06,2.63794e+06,77411,439943,769133,201801,ahorro
4,1263145.0,461652294130885802,1,230,230029115177,2018-01-31,2012-03-14,Activa,7.98233e+06,2.16149e+06,1.12837e+06,5.63678e+06,6819,3.20501e+06,36452,201801,ahorro


In [8]:
tabla_final.count()

sk_cliente                  21034062
id_cliente                  21022271
tipo_id                     21034096
cod_producto                21034096
nro_cuenta                  21034096
fecha_ultima_transaccion    20427650
fecha_apertura              21033978
estado                      21034096
saldo                       21034096
promedio1                   21034096
promedio2                   21034096
promedio3                   21034096
promedio4                   21034096
promedio5                   21034096
promedio6                   21034096
periodo                     21034096
tipo_producto               21034096
dtype: int64

In [9]:
tabla_final.tail()

,sk_cliente,id_cliente,tipo_id,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,tipo_producto
21060168,2483365.0,725955242634580402,1,230,230470202490,2020-03-26,2019-03-06,Activa,19455781,2,3769114,109493,297335,152483,8122,202003,ahorro
21060169,1361952.0,478552295443431501,1,230,230220785950,2020-03-25,1998-07-28,Activa,1922,81,23821,28979,7806,62840,7912,202003,ahorro
21060170,2756764.0,862252308256137301,1,230,230251018529,2020-03-25,2005-03-15,Activa,1376359,4,313354,151841,53414,738885,319235,202003,ahorro
21060171,3115603.0,881452305336560101,1,300,30018002817195,1997-03-07,2020-02-19,Depositado,1500000,1500000,0,0,0,0,0,202003,cdt
21060172,3115603.0,881452305336560101,1,300,30018002822823,1997-03-07,2015-10-06,Depositado,600000,600000,0,0,0,0,0,202003,cdt


In [10]:
tabla_final.dtypes

sk_cliente                         float64
id_cliente                          object
tipo_id                              int64
cod_producto                         int64
nro_cuenta                          object
fecha_ultima_transaccion    datetime64[ns]
fecha_apertura              datetime64[ns]
estado                              object
saldo                               object
promedio1                           object
promedio2                           object
promedio3                           object
promedio4                           object
promedio5                           object
promedio6                           object
periodo                              int64
tipo_producto                       object
dtype: object

In [11]:
tabla_final.groupby('periodo').size()

periodo
201801    753963
201802    765427
201803    773770
201804    779901
201805    787812
201806    789579
201807    791780
201808    798526
201809    801608
201810    800947
201811    801734
201812    801233
201901    796916
201902    794192
201903    790678
201904    794084
201905    774032
201906    778347
201907    772755
201908    765527
201909    761364
201910    755992
201911    786453
201912    775085
202001    758691
202002    748443
202003    735257
dtype: int64

In [12]:
## eliminando comas:

tabla_final['saldo'] = [float(str(x).replace(',','.')) for x in tabla_final['saldo']]
tabla_final['promedio1'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio1']]
tabla_final['promedio2'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio2']]
tabla_final['promedio3'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio3']]
tabla_final['promedio4'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio4']]
tabla_final['promedio5'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio5']]
tabla_final['promedio6'] = [float(str(x).replace(',','.')) for x in tabla_final['promedio6']]


In [13]:
## Cast variables

tabla_final = tabla_final[tabla_final['id_cliente'].notnull()]

tabla_final['id_cliente'] = tabla_final['id_cliente'].astype('int64')
tabla_final['nro_cuenta'] = tabla_final['nro_cuenta'].astype('int64')

tabla_final['saldo'] = tabla_final['saldo'].astype('float64')
tabla_final['promedio1'] = tabla_final['promedio1'].astype('float64')
tabla_final['promedio2'] = tabla_final['promedio2'].astype('float64')
tabla_final['promedio3'] = tabla_final['promedio3'].astype('float64')
tabla_final['promedio4'] = tabla_final['promedio4'].astype('float64')
tabla_final['promedio5'] = tabla_final['promedio5'].astype('float64')
tabla_final['promedio6'] = tabla_final['promedio6'].astype('float64')
tabla_final['fecha_apertura'] = pd.to_datetime(tabla_final['fecha_apertura'], format='%Y-%m-%d')
tabla_final['fecha_ultima_transaccion'] = pd.to_datetime(tabla_final['fecha_ultima_transaccion'], format='%Y-%m-%d')


In [14]:
tabla_final[tabla_final['periodo'] == 201907].count()

sk_cliente                  768667
id_cliente                  768667
tipo_id                     768667
cod_producto                768667
nro_cuenta                  768667
fecha_ultima_transaccion    744302
fecha_apertura              768663
estado                      768667
saldo                       768667
promedio1                   768667
promedio2                   768667
promedio3                   768667
promedio4                   768667
promedio5                   768667
promedio6                   768667
periodo                     768667
tipo_producto               768667
dtype: int64

In [15]:
tabla_final.dtypes

sk_cliente                         float64
id_cliente                           int64
tipo_id                              int64
cod_producto                         int64
nro_cuenta                           int64
fecha_ultima_transaccion    datetime64[ns]
fecha_apertura              datetime64[ns]
estado                              object
saldo                              float64
promedio1                          float64
promedio2                          float64
promedio3                          float64
promedio4                          float64
promedio5                          float64
promedio6                          float64
periodo                              int64
tipo_producto                       object
dtype: object

In [16]:
tabla_final[tabla_final['periodo'] == 201908].head()

,sk_cliente,id_cliente,tipo_id,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,tipo_producto
14965953,1.0,673152298768285201,1,210,210550400055,2019-08-30,2002-02-12,Activa,13331029.02,11123634.0,11585101.0,8585039.0,8190562.0,9364111.0,7935859.0,201908,ahorro
14965954,1.0,203652298748593302,1,230,230550770804,2019-08-27,1998-05-04,Activa,14098745.78,8675454.0,4708799.0,5500841.0,5272105.0,5158556.0,4736269.0,201908,ahorro
14965955,1.0,503352308607651302,1,230,230550094650,2019-08-30,2004-04-12,Activa,1232538.84,246643.0,272576.0,639295.0,201754.0,76439.0,143057.0,201908,ahorro
14965956,1.0,367352310001575001,1,230,230065113821,2019-08-01,2011-04-19,Activa,2542.74,2542.0,20843.0,6200.0,27436.0,1486.0,82554.0,201908,ahorro
14965957,1.0,538452310932351901,1,230,230540165610,2019-09-03,2012-07-04,Activa,657841.07,396067.0,1049388.0,742042.0,1849075.0,2880502.0,2514067.0,201908,ahorro


In [17]:
tabla_final = tabla_final[tabla_final['id_cliente'] != -1]

In [18]:
tabla_final['fecha'] = pd.to_datetime(tabla_final['periodo'], format='%Y%m')

In [20]:
path_in="s3://adl-refined-dev-popular/data_orig/productos/pasivo/"

tabla_final.to_parquet(path_in+"pasivos_201801_202003", engine='pyarrow',index=False)

# Cargando Data

In [ ]:
path_in="s3://adl-refined-dev-popular/data_orig/productos/pasivo/"

pasivo_201803_202003 = pd.read_parquet(path_in+"pasivos_201801_202003", engine='pyarrow')

In [ ]:
## Validación de duplicados a nivel 'id_cliente','nrooblig','fecha'

In [21]:
pasivo_201803_202003=tabla_final

In [22]:
pasivo_201803_202003=pasivo_201803_202003.sort_values(['id_cliente','nro_cuenta','periodo'])

In [23]:
#Se eliminan duplicados
pasivo_201803_202003=pasivo_201803_202003.drop_duplicates(subset=['id_cliente','nro_cuenta','periodo'],keep='first')

In [24]:
pasivo_201803_202003.shape

(21009137, 18)

In [25]:
pasivo_201803_202003['fecha_pasivo']=pasivo_201803_202003['periodo'].astype(int)

In [ ]:
##Creacion variables SMLV

In [26]:
pasivo_201803_202003['ano_po'] = pasivo_201803_202003['periodo'].astype('str').str[0:4].astype(int)

#Se asigna el salario minimo mensual vigente de 2018 y 2019
pasivo_201803_202003['SMLV'] = np.where(pasivo_201803_202003['ano_po'] == 2018, 781242,
                                        np.where(pasivo_201803_202003['ano_po'] == 2019, 828116,
                                                 np.where(pasivo_201803_202003['ano_po'] == 2020, 877803,
                                                          np.nan)))


In [27]:
pasivo_201803_202003['dia_fecha_corte'] = pasivo_201803_202003['fecha']+timedelta(days=37) ##TODO: Confirmar días


In [28]:
pasivo_201803_202003

,sk_cliente,id_cliente,tipo_id,cod_producto,nro_cuenta,fecha_ultima_transaccion,fecha_apertura,estado,saldo,promedio1,promedio2,promedio3,promedio4,promedio5,promedio6,periodo,tipo_producto,fecha,fecha_pasivo,ano_po,SMLV,dia_fecha_corte
579698,440856.0,101052294083942902,3,110,110180211385,2018-02-01,1992-04-01,Activa,43915433.50,43151218.16,38360511.39,34139410.17,32022367.39,31188060.37,20473727.32,201801,corriente,2018-01-01,201801,2018,781242.0,2018-02-07
1339668,440856.0,101052294083942902,3,110,110180211385,2018-03-01,1992-04-01,Activa,36178817.50,47625900.64,43151218.16,38360511.39,34139410.17,32022367.39,31188060.37,201802,corriente,2018-02-01,201802,2018,781242.0,2018-03-10
2105450,440856.0,101052294083942902,3,110,110180211385,2018-04-02,1992-04-01,Activa,52408828.50,52980041.10,47625900.64,43151218.16,38360511.39,34139410.17,32022367.39,201803,corriente,2018-03-01,201803,2018,781242.0,2018-04-07
2882595,440856.0,101052294083942902,3,110,110180211385,2018-05-02,1992-04-01,Activa,80046970.50,65801499.27,52980041.10,47625900.64,43151218.16,38360511.39,34139410.17,201804,corriente,2018-04-01,201804,2018,781242.0,2018-05-08
3666648,440856.0,101052294083942902,3,110,110180211385,2018-06-05,1992-04-01,Activa,15583774.50,40659199.74,65801499.27,52980041.10,47625900.64,43151218.16,38360511.39,201805,corriente,2018-05-01,201805,2018,781242.0,2018-06-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17545168,2894827.0,999955266612957502,1,210,210460344260,2019-12-02,2004-12-21,Activa,967.51,1732.00,16899.00,3163.00,11466.00,45517.00,18576.00,201911,ahorro,2019-11-01,201911,2019,828116.0,2019-12-08
18326562,2894827.0,999955266612957502,1,210,210460344260,2019-12-05,2004-12-21,Activa,826.70,91798.00,1732.00,16899.00,3163.00,11466.00,45517.00,201912,ahorro,2019-12-01,201912,2019,828116.0,2020-01-07
19093539,2894827.0,999955266612957502,1,210,210460344260,2020-02-03,2004-12-21,Activa,2569.72,14448.00,91798.00,1732.00,16899.00,3163.00,11466.00,202001,ahorro,2020-01-01,202001,2020,877803.0,2020-02-07
19847986,2894827.0,999955266612957502,1,210,210460344260,2020-03-02,2004-12-21,Activa,6199.72,5949.00,14448.00,91798.00,1732.00,16899.00,3163.00,202002,ahorro,2020-02-01,202002,2020,877803.0,2020-03-09


### Creación base a nivel id_cliente, fecha

In [29]:
pasivo_201803_202003_cli_fec=pd.DataFrame(pasivo_201803_202003.groupby(['id_cliente','periodo'])[['SMLV','fecha_pasivo']].max())

In [30]:
pasivo_201803_202003_cli_fec.shape

(19480816, 2)

### Variables Pasivo

In [ ]:
# Número de cuentas totales vigentes al punto de observación
# Número de cuentas vigentes al punto de observación tipo cta Y
# Dias desde la última transacción
# Antigüedad del cliente con ctas del pasivo
# meses desde la última apertura de cta
# Saldo total a fin de mes
# Saldo maximo a fin de mes
# Saldo mínimo a fin de mes
# Saldo total x meses atrás
# Saldo promedio últimos x meses
# Saldo promedio últimos x meses por tipo cta Y
# tipo de cuenta Digital, nueva, etc.


In [31]:
pasivo_201803_202003.columns

Index(['sk_cliente', 'id_cliente', 'tipo_id', 'cod_producto', 'nro_cuenta',
       'fecha_ultima_transaccion', 'fecha_apertura', 'estado', 'saldo',
       'promedio1', 'promedio2', 'promedio3', 'promedio4', 'promedio5',
       'promedio6', 'periodo', 'tipo_producto', 'fecha', 'fecha_pasivo',
       'ano_po', 'SMLV', 'dia_fecha_corte'],
      dtype='object')

In [32]:
## Agrupaciones
gp_pasv = pasivo_201803_202003.groupby(['id_cliente','periodo'])
gp_pasv_cc = pasivo_201803_202003[pasivo_201803_202003['tipo_producto']=='corriente'].groupby(['id_cliente','periodo'])
gp_pasv_ca = pasivo_201803_202003[pasivo_201803_202003['tipo_producto']=='ahorro'].groupby(['id_cliente','periodo'])
gp_pasv_cdt = pasivo_201803_202003[pasivo_201803_202003['tipo_producto']=='cdt'].groupby(['id_cliente','periodo'])

In [33]:
## Número de cuentas totales vigentes al punto de observación
pasivo_201803_202003_cli_fec['pasv_num_ctas_vig_tot'] = gp_pasv['nro_cuenta'].count()
## Número de cuentas totales vigentes al punto de observación por tipo cta
pasivo_201803_202003_cli_fec['pasv_num_ctas_vig_cc'] = gp_pasv['tipo_producto'].value_counts().unstack()['corriente'].fillna(0).astype(int)
pasivo_201803_202003_cli_fec['pasv_num_ctas_vig_ca'] = gp_pasv['tipo_producto'].value_counts().unstack()['ahorro'].fillna(0).astype(int)
pasivo_201803_202003_cli_fec['pasv_num_ctas_vig_cdt'] = gp_pasv['tipo_producto'].value_counts().unstack()['cdt'].fillna(0).astype(int)
## Dia última transacción
pasivo_201803_202003_cli_fec['pasv_dias_desde_ultima_trans'] = (gp_pasv['dia_fecha_corte'].max() - 
                                                                gp_pasv['fecha_ultima_transaccion'].max())/np.timedelta64(1, 'D')
## Antiguedad Total
pasivo_201803_202003_cli_fec['pasv_antig_total'] = (gp_pasv['dia_fecha_corte'].max() - 
                                                                gp_pasv['fecha_apertura'].min())/np.timedelta64(30, 'D')
## Numero meses ultima apertura
pasivo_201803_202003_cli_fec['pasv_num_meses_ult_apertura'] = (gp_pasv['dia_fecha_corte'].max() - 
                                                                gp_pasv['fecha_apertura'].max())/np.timedelta64(30, 'D')
## Saldo Total por cartera fin de mes
pasivo_201803_202003_cli_fec['pasv_saldo_tot_fin_mes_smlv'] = (gp_pasv['saldo'].sum())/(gp_pasv['SMLV'].max())
pasivo_201803_202003_cli_fec['pasv_saldo_cc_fin_mes_smlv'] = (gp_pasv_cc['saldo'].sum())/(gp_pasv_cc['SMLV'].max())
pasivo_201803_202003_cli_fec['pasv_saldo_ca_fin_mes_smlv'] = (gp_pasv_ca['saldo'].sum())/(gp_pasv_ca['SMLV'].max())
pasivo_201803_202003_cli_fec['pasv_saldo_cdt_fin_mes_smlv'] = (gp_pasv_cdt['saldo'].sum())/(gp_pasv_cdt['SMLV'].max())
## Saldo Min fin de mes
pasivo_201803_202003_cli_fec['pasv_saldo_min_fin_mes_smlv'] = (gp_pasv['saldo'].min())/(gp_pasv['SMLV'].max())
pasivo_201803_202003_cli_fec['pasv_saldo_min_cc_fin_mes_smlv'] = (gp_pasv_cc['saldo'].min())/(gp_pasv_cc['SMLV'].max())
pasivo_201803_202003_cli_fec['pasv_saldo_min_ca_fin_mes_smlv'] = (gp_pasv_ca['saldo'].min())/(gp_pasv_ca['SMLV'].max())
pasivo_201803_202003_cli_fec['pasv_saldo_min_cdt_fin_mes_smlv'] = (gp_pasv_cdt['saldo'].min())/(gp_pasv_cdt['SMLV'].max())
## Saldo Max fin de mes
pasivo_201803_202003_cli_fec['pasv_saldo_max_fin_mes_smlv'] = (gp_pasv['saldo'].max())/(gp_pasv['SMLV'].max())
pasivo_201803_202003_cli_fec['pasv_saldo_max_cc_fin_mes_smlv'] = (gp_pasv_cc['saldo'].max())/(gp_pasv_cc['SMLV'].max())
pasivo_201803_202003_cli_fec['pasv_saldo_max_ca_fin_mes_smlv'] = (gp_pasv_ca['saldo'].max())/(gp_pasv_ca['SMLV'].max())
pasivo_201803_202003_cli_fec['pasv_saldo_max_cdt_fin_mes_smlv'] = (gp_pasv_cdt['saldo'].max())/(gp_pasv_cdt['SMLV'].max())

# Reset index
pasivo_201803_202003_cli_fec.reset_index(inplace=True)
# Creando fecha
pasivo_201803_202003_cli_fec['fecha'] = pd.to_datetime(pasivo_201803_202003_cli_fec['periodo'], format='%Y%m')

KeyboardInterrupt: 

In [ ]:
pasivo_201803_202003_cli_fec.head()

In [ ]:
def gp_func(ids, date):
    fecha = pd.to_datetime(date, format='%Y%m')
    n_meses = [1,3,6]
    
    pasv_out = pd.DataFrame(index=ids) # this is different from the actual prod code
    
    for n in n_meses:
        col_name = 'pasv_saldo_total_'+str(n)+'mes_atras'
        col_name_cc = 'pasv_saldo_cc_'+str(n)+'mes_atras'
        col_name_ca = 'pasv_saldo_ca_'+str(n)+'mes_atras'
        col_name_cdt = 'pasv_saldo_cdt_'+str(n)+'mes_atras'
        date_past = fecha - timedelta(days=n*30)

        gp = pasivo_201803_202003_cli_fec[(pasivo_201803_202003_cli_fec['fecha']>date_past)&(pasivo_201803_202003_cli_fec['fecha']<=fecha)].groupby('id_cliente')
        
        pasv_out[col_name] = gp['pasv_saldo_tot_fin_mes_smlv'].sum()/n
        pasv_out[col_name_cc] = gp['pasv_saldo_cc_fin_mes_smlv'].sum()/n
        pasv_out[col_name_ca] = gp['pasv_saldo_ca_fin_mes_smlv'].sum()/n
        pasv_out[col_name_cdt] = gp['pasv_saldo_cdt_fin_mes_smlv'].sum()/n
        
    pasv_out['periodo'] = int(date)
    pasv_out.index.rename('id_cliente', inplace=True)
    pasv_out.reset_index(inplace=True)
    
    return pasv_out

In [ ]:
# Reset index
pasivo_201803_202003_cli_fec.reset_index(inplace=True)

In [ ]:
str_dates = ['201801','201802','201803','201804','201805','201806','201807','201808','201809','201810','201811','201812',
            '201901','201902','201903','201904','201905','201906','201907','201908','201909','201910','201911','201912',
            '202001','202002','202003']

for ix, date in enumerate(str_dates):
    print(date)
    ids = pasivo_201803_202003_cli_fec.loc[pasivo_201803_202003_cli_fec['periodo']==int(date), 'id_cliente'].unique()
    
    if ix == 0:
        pasv_out = gp_func(ids, date)
    else:    
        pasv_temp = gp_func(ids, date)
        pasv_out = pd.concat([pasv_out, pasv_temp], ignore_index=True)

        
## Set_index
pasivo_201803_202003_cli_fec.set_index(['id_cliente','periodo'], inplace=True)
pasv_out.set_index(['id_cliente','periodo'], inplace=True)

In [ ]:
pasivo_201803_202003_cli_fec.head(20)

In [ ]:
pasv_out.tail()

In [ ]:
pasivo_201803_202003_cli_fec_tot = pasivo_201803_202003_cli_fec.join(pasv_out, how='left')
print(pasv_out.shape)
print(pasivo_201803_202003_cli_fec.shape)
print(pasivo_201803_202003_cli_fec_tot.shape)

pasivo_201803_202003_cli_fec_tot.head()

In [ ]:
### se calculan variables a nivel obligacion 
pasivo_201803_201908['fecha_ultima_transaccion_2']=np.where(pasivo_201803_201908['fecha_ultima_transaccion']==0,np.nan,
                                                   np.where(~pasivo_201803_201908['fecha'].isin(['201904','201908','201909']),pasivo_201803_201908['fecha_ultima_transaccion'].str[6:10]+pasivo_201803_201908['fecha_ultima_transaccion'].str[3:5],pasivo_201803_201908['fecha_ultima_transaccion'].astype('str').str[0:6]))
pasivo_201803_201908['fecha_apertura_2']=np.where(~pasivo_201803_201908['fecha'].isin(['201904','201908','201909']),pasivo_201803_201908['fecha_apertura'].str[6:10]+pasivo_201803_201908['fecha_apertura'].str[3:5],pasivo_201803_201908['fecha_apertura'].astype('str').str[0:6])

In [ ]:
### se calculan variables a nivel obligacion 
pasivo_201803_201908['edad_pas']=(pasivo_201803_201908['fecha'].str[0:4].astype('float')-pasivo_201803_201908['fecha_apertura_2'].str[0:4].astype('float'))*12+(pasivo_201803_201908['fecha'].str[4:7].astype('float')-pasivo_201803_201908['fecha_apertura_2'].str[4:7].astype('float'))
pasivo_201803_201908['edad_trans_pas']=(pasivo_201803_201908['fecha'].str[0:4].astype('float')-pasivo_201803_201908['fecha_ultima_transaccion_2'].str[0:4].astype('float'))*12+(pasivo_201803_201908['fecha'].str[4:7].astype('float')-pasivo_201803_201908['fecha_ultima_transaccion_2'].str[4:7].astype('float'))

In [ ]:
#Marca de apertura realizada en los ultimos x meses
pasivo_201803_201908['apertura_1meses']=np.where((pasivo_201803_201908['edad_pas']<=1),1,0)
pasivo_201803_201908['apertura_3meses']=np.where((pasivo_201803_201908['edad_pas']<=3),1,0)
pasivo_201803_201908['apertura_6meses']=np.where((pasivo_201803_201908['edad_pas']<=6),1,0)
pasivo_201803_201908['apertura_12meses']=np.where((pasivo_201803_201908['edad_pas']<=12),1,0)

In [ ]:
# Numero de aperturas total ultimos 3, 6, 9 y 12 meses por tipo de pasivo
antiguedad=['1','3','6','12']
cartera=['230','210','300','110','280','220','250','260','240']
def num_aper_tot(antiguedad,cartera):
    pasivo_201803_201908_cli_fec['num_aper_ultim_'+antiguedad+'meses_'+cartera] = pasivo_201803_201908[pasivo_201803_201908['cod_producto'].astype(str)==cartera].groupby(['id_cliente','fecha'])[['apertura_'+antiguedad+'meses']].sum()
    pasivo_201803_201908_cli_fec.fillna({'num_aper_ultim_'+antiguedad+'meses_'+cartera : 0}, inplace = True)
    
for ant in antiguedad:
    for car in cartera:
        num_aper_tot(ant,car)


In [ ]:
# Numero de aperturas total ultimos 3, 6, 9 y 12 meses por tipo de pasivo
antiguedad=['1','3','6','12']

def num_aper_tot_tot(antiguedad):
    pasivo_201803_201908_cli_fec['num_aper_ultim_'+antiguedad+'meses_tot'] = pasivo_201803_201908.groupby(['id_cliente','fecha'])[['apertura_'+antiguedad+'meses']].sum()
    pasivo_201803_201908_cli_fec.fillna({'num_aper_ultim_'+antiguedad+'meses_tot' : 0}, inplace = True)
    
for ant in antiguedad:
    num_aper_tot_tot(ant)

In [ ]:
pasivo_201803_201908_cli_fec['num_aper_ultim_1meses_tot_pas']=pasivo_201803_201908_cli_fec['num_aper_ultim_1meses_tot']
pasivo_201803_201908_cli_fec['num_aper_ultim_3meses_tot_pas']=pasivo_201803_201908_cli_fec['num_aper_ultim_3meses_tot']
pasivo_201803_201908_cli_fec['num_aper_ultim_6meses_tot_pas']=pasivo_201803_201908_cli_fec['num_aper_ultim_6meses_tot']
pasivo_201803_201908_cli_fec['num_aper_ultim_12meses_tot_pas']=pasivo_201803_201908_cli_fec['num_aper_ultim_12meses_tot']

In [ ]:
del pasivo_201803_201908_cli_fec['num_aper_ultim_1meses_tot']
del pasivo_201803_201908_cli_fec['num_aper_ultim_3meses_tot']
del pasivo_201803_201908_cli_fec['num_aper_ultim_6meses_tot']
del pasivo_201803_201908_cli_fec['num_aper_ultim_12meses_tot']

### Variables meses desde la última transacción

In [ ]:
#Funcion que calcula los meses desde la ultima apertura por tipo pasivo
cartera=['230','210','300','110','280','220','250','260','240']
def meses_ult_aper(cartera):
    pasivo_201803_201908_cli_fec['meses_ultim_trans_'+cartera] = pasivo_201803_201908[pasivo_201803_201908['cod_producto'].astype(str)==cartera].groupby(['id_cliente','fecha'])[['edad_trans_pas']].min()

for car in cartera:
    meses_ult_aper(car)

In [ ]:
pasivo_201803_201908_cli_fec['meses_ultim_trans_tot'] = pasivo_201803_201908.groupby(['id_cliente','fecha'])[['edad_trans_pas']].min() 

### Antiguedad en el pasivo

In [ ]:
pasivo_201803_201908_cli_fec['antiguedad_tot_pas'] = pasivo_201803_201908.groupby(['id_cliente','fecha'])[['edad_pas']].max()   

### Número de obligaciones vigentes

In [ ]:
# Numero de obligaciones diferentes carteras y estados a la fecha del reporte
#Estados que aplican para cdt
estados=['Depositado','Exigible']
#Estados que aplica para todos menos cdt
estado2=['Activa','Nueva']
#Carteras sin cdt
cartera=['230','210','110','280','220','250','260','240']

def num_estado_cartera(cartera,estado):
    pasivo_201803_201908_cli_fec['num_obl_vig_'+cartera+'_'+estado] = pasivo_201803_201908[(pasivo_201803_201908['cod_producto'].astype(str)==cartera) & (pasivo_201803_201908['estado']==estado)].groupby(['id_cliente','fecha'])[['cod_producto']].count()
    pasivo_201803_201908_cli_fec.fillna({'num_obl_vig_'+cartera+'_'+estado : 0}, inplace = True)

for car in cartera:
    for es in estado2:
        num_estado_cartera(car,es)
        
#Para cdt(300) solo aplica depositado y exigible
num_estado_cartera('300','Depositado')
num_estado_cartera('300','Exigible')

In [ ]:
# Numero de obligaciones vigentes de diferentes  estados a la fecha del reporte
estados=['Activa','Depositado','Nueva','Exigible']

def num_estado(estado):    
    pasivo_201803_201908_cli_fec['num_obl_vig_'+estado] = pasivo_201803_201908[(pasivo_201803_201908['estado']==estado)].groupby(['id_cliente','fecha'])[['estado']].count()  
    pasivo_201803_201908_cli_fec.fillna({'num_obl_vig_'+estado : 0}, inplace = True)

for es in estados:
    num_estado(es)

In [ ]:
#Se guarda la base hasta este punto
#Se nombra como activos_201803_201907_cli_fec porque incluye desde marzo 201803
pasivo_201803_201908_cli_fec.to_parquet("/mnt/work/CU_Cobranzas/ModelVersion1/Notebooks/Data construida/pasivo_201803_201908_cli_fec",engine='pyarrow')

In [ ]:
# Numero de obligaciones vigentes de diferentes carteras  a la fecha del reporte
cartera=['230','210','300','110','280','220','250','260','240']

def num_car(cartera):    
    pasivo_201803_201908_cli_fec['num_obl_vig_'+cartera] = pasivo_201803_201908[(pasivo_201803_201908['cod_producto'].astype(str)==cartera)].groupby(['id_cliente','fecha'])[['cod_producto']].count()  
    pasivo_201803_201908_cli_fec.fillna({'num_obl_vig_'+cartera : 0}, inplace = True)

for car in cartera:
    num_car(car)

In [ ]:
# número de obligaciones pasivo total
pasivo_201803_201908_cli_fec['num_obl_pasivo'] = pasivo_201803_201908.groupby(['id_cliente','fecha'])[['cod_producto']].count()  
pasivo_201803_201908_cli_fec.fillna({'num_obl_pasivo' : 0}, inplace = True)

In [ ]:
#Tiene pasivo no tiene pasivo
pasivo_201803_201908_cli_fec['tiene_pasivo'] = np.where(pasivo_201803_201908_cli_fec['num_obl_pasivo']>0,1,0)

### Variables saldo

In [ ]:
cartera=['230','210','300','110','280','220','250','260','240']
def saldo(cartera):

    pasivo_201803_201908_cli_fec['saldo_tot_'+cartera] = pasivo_201803_201908[ (pasivo_201803_201908['cod_producto'].astype(str)==cartera)].groupby(['id_cliente','fecha'])[['saldo']].sum()
  
    pasivo_201803_201908_cli_fec['saldo_max_'+cartera] = pasivo_201803_201908[ (pasivo_201803_201908['cod_producto'].astype(str)==cartera)].groupby(['id_cliente','fecha'])[['saldo']].max()
   
    pasivo_201803_201908_cli_fec['saldo_min_'+cartera] = pasivo_201803_201908[ (pasivo_201803_201908['cod_producto'].astype(str)==cartera)].groupby(['id_cliente','fecha'])[['saldo']].min()
   
    pasivo_201803_201908_cli_fec['saldo_prom_'+cartera] = pasivo_201803_201908[ (pasivo_201803_201908['cod_producto'].astype(str)==cartera)].groupby(['id_cliente','fecha'])[['saldo']].mean()
   
    pasivo_201803_201908_cli_fec['saldo_prom_ult1mes'+cartera] = pasivo_201803_201908[ (pasivo_201803_201908['cod_producto'].astype(str)==cartera)].groupby(['id_cliente','fecha'])[['promedio1']].mean()
    pasivo_201803_201908_cli_fec['saldo_prom_ult2mes'+cartera] = pasivo_201803_201908[ (pasivo_201803_201908['cod_producto'].astype(str)==cartera)].groupby(['id_cliente','fecha'])[['promedio2']].mean()
    pasivo_201803_201908_cli_fec['saldo_prom_ult3mes'+cartera] = pasivo_201803_201908[ (pasivo_201803_201908['cod_producto'].astype(str)==cartera)].groupby(['id_cliente','fecha'])[['promedio3']].mean()
    pasivo_201803_201908_cli_fec['saldo_prom_ult4mes'+cartera] = pasivo_201803_201908[ (pasivo_201803_201908['cod_producto'].astype(str)==cartera)].groupby(['id_cliente','fecha'])[['promedio4']].mean()
    pasivo_201803_201908_cli_fec['saldo_prom_ult5mes'+cartera] = pasivo_201803_201908[ (pasivo_201803_201908['cod_producto'].astype(str)==cartera)].groupby(['id_cliente','fecha'])[['promedio5']].mean()
    pasivo_201803_201908_cli_fec['saldo_prom_ult6mes'+cartera] = pasivo_201803_201908[ (pasivo_201803_201908['cod_producto'].astype(str)==cartera)].groupby(['id_cliente','fecha'])[['promedio6']].mean()
   
for car in cartera:
    saldo(car) 


In [ ]:
#total
pasivo_201803_201908_cli_fec['saldo_tot_pas_tot'] = pasivo_201803_201908.groupby(['id_cliente','fecha'])[['saldo']].sum()
pasivo_201803_201908_cli_fec['saldo_max_pas_tot'] = pasivo_201803_201908.groupby(['id_cliente','fecha'])[['saldo']].max()
pasivo_201803_201908_cli_fec['saldo_min_pas_tot'] = pasivo_201803_201908.groupby(['id_cliente','fecha'])[['saldo']].min()
pasivo_201803_201908_cli_fec['saldo_prom_pas_tot'] = pasivo_201803_201908.groupby(['id_cliente','fecha'])[['saldo']].mean()
pasivo_201803_201908_cli_fec['saldo_prom_pas_ult1mes'] = pasivo_201803_201908.groupby(['id_cliente','fecha'])[['promedio1']].mean()
pasivo_201803_201908_cli_fec['saldo_prom_pas_ult2mes'] = pasivo_201803_201908.groupby(['id_cliente','fecha'])[['promedio2']].mean()
pasivo_201803_201908_cli_fec['saldo_prom_pas_ult3mes'] = pasivo_201803_201908.groupby(['id_cliente','fecha'])[['promedio3']].mean()
pasivo_201803_201908_cli_fec['saldo_prom_pas_ult4mes'] = pasivo_201803_201908.groupby(['id_cliente','fecha'])[['promedio4']].mean()
pasivo_201803_201908_cli_fec['saldo_prom_pas_ult5mes'] = pasivo_201803_201908.groupby(['id_cliente','fecha'])[['promedio5']].mean()
pasivo_201803_201908_cli_fec['saldo_prom_pas_ult6mes'] = pasivo_201803_201908.groupby(['id_cliente','fecha'])[['promedio6']].mean()
 

In [ ]:
### Creación variables saldo a salario minimos

In [ ]:
variables_saldo_cupo_cuota=['saldo_tot_230',
'saldo_max_230',
'saldo_min_230',
'saldo_prom_230',
'saldo_prom_ult1mes230',
'saldo_prom_ult2mes230',
'saldo_prom_ult3mes230',
'saldo_prom_ult4mes230',
'saldo_prom_ult5mes230',
'saldo_prom_ult6mes230',
'saldo_tot_210',
'saldo_max_210',
'saldo_min_210',
'saldo_prom_210',
'saldo_prom_ult1mes210',
'saldo_prom_ult2mes210',
'saldo_prom_ult3mes210',
'saldo_prom_ult4mes210',
'saldo_prom_ult5mes210',
'saldo_prom_ult6mes210',
'saldo_tot_300',
'saldo_max_300',
'saldo_min_300',
'saldo_prom_300',
'saldo_prom_ult1mes300',
'saldo_prom_ult2mes300',
'saldo_prom_ult3mes300',
'saldo_prom_ult4mes300',
'saldo_prom_ult5mes300',
'saldo_prom_ult6mes300',
'saldo_tot_110',
'saldo_max_110',
'saldo_min_110',
'saldo_prom_110',
'saldo_prom_ult1mes110',
'saldo_prom_ult2mes110',
'saldo_prom_ult3mes110',
'saldo_prom_ult4mes110',
'saldo_prom_ult5mes110',
'saldo_prom_ult6mes110',
'saldo_tot_280',
'saldo_max_280',
'saldo_min_280',
'saldo_prom_280',
'saldo_prom_ult1mes280',
'saldo_prom_ult2mes280',
'saldo_prom_ult3mes280',
'saldo_prom_ult4mes280',
'saldo_prom_ult5mes280',
'saldo_prom_ult6mes280',
'saldo_tot_220',
'saldo_max_220',
'saldo_min_220',
'saldo_prom_220',
'saldo_prom_ult1mes220',
'saldo_prom_ult2mes220',
'saldo_prom_ult3mes220',
'saldo_prom_ult4mes220',
'saldo_prom_ult5mes220',
'saldo_prom_ult6mes220',
'saldo_tot_250',
'saldo_max_250',
'saldo_min_250',
'saldo_prom_250',
'saldo_prom_ult1mes250',
'saldo_prom_ult2mes250',
'saldo_prom_ult3mes250',
'saldo_prom_ult4mes250',
'saldo_prom_ult5mes250',
'saldo_prom_ult6mes250',
'saldo_tot_260',
'saldo_max_260',
'saldo_min_260',
'saldo_prom_260',
'saldo_prom_ult1mes260',
'saldo_prom_ult2mes260',
'saldo_prom_ult3mes260',
'saldo_prom_ult4mes260',
'saldo_prom_ult5mes260',
'saldo_prom_ult6mes260',
'saldo_tot_240',
'saldo_max_240',
'saldo_min_240',
'saldo_prom_240',
'saldo_prom_ult1mes240',
'saldo_prom_ult2mes240',
'saldo_prom_ult3mes240',
'saldo_prom_ult4mes240',
'saldo_prom_ult5mes240',
'saldo_prom_ult6mes240',
'saldo_tot_pas_tot',
'saldo_max_pas_tot',
'saldo_min_pas_tot',
'saldo_prom_pas_tot',
'saldo_prom_pas_ult1mes',
'saldo_prom_pas_ult2mes',
'saldo_prom_pas_ult3mes',
'saldo_prom_pas_ult4mes',
'saldo_prom_pas_ult5mes',
'saldo_prom_pas_ult6mes']

In [ ]:
#Funcion que vuelve la variable en salarios minimos mensuales vigentes
def to_smlv(base,variable):
    
    base[variable+'_SMLV']=base[variable]/base['SMLV']
    del base[variable]
    
for var in variables_saldo_cupo_cuota:
    to_smlv(pasivo_201803_201908_cli_fec,var)   

In [ ]:
pasivo_201803_201908_cli_fec.shape

## Exporta la base

In [ ]:
pasivo_201803_202003_cli_fec_tot.describe()

In [ ]:
pasivo_201803_202003_cli_fec_tot.drop(columns='index', inplace=True)

In [ ]:
pasivo_201803_202003_cli_fec_tot.reset_index(inplace=True)
pasivo_201803_202003_cli_fec_tot.head()

In [ ]:
pasivo_201803_202003_cli_fec_tot.to_parquet("s3://adl-refined-dev-popular/parquet/TC_adquisicion/pasivo_201801_202003",engine='pyarrow', index=False)
